In [1]:
import os
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from pycocotools.coco import COCO
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import os
import json

# Define paths
annotations_file = r'C:/Users/ADVAIT/Desktop/CarDD_release/CarDD_release/CarDD_COCO/annotations/instances_train2017.json'  # Adjust the path
images_dir =  r'C:/Users/ADVAIT/Desktop/CarDD_release/CarDD_release/CarDD_COCO/train2017'

# Load the COCO annotations
with open(annotations_file) as f:
    coco_data = json.load(f)

# Function to load images and labels
def load_data(coco_data):
    images = []
    labels = []
    for image in coco_data['images']:
        img_path = os.path.join(images_dir, image['file_name'])
        images.append(img_path)
        # Create a label mapping (this is a simplified example)
        img_labels = [ann['category_id'] for ann in coco_data['annotations'] if ann['image_id'] == image['id']]
        labels.append(img_labels)
    return images, labels

images, labels = load_data(coco_data)


In [3]:
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [600, 600])  # Resize to a suitable input size
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Apply preprocessing to your dataset
preprocessed_images = [preprocess_image(img) for img in images]


In [4]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Define the path to the model directory
PATH_TO_MODEL_DIR = r"C:/Users/ADVAIT/Desktop/CrashCal/models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8"  # Update this line with your folder name

# Load the pipeline configuration
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
config_path = f"{PATH_TO_MODEL_DIR}/pipeline.config"

with tf.io.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

# Load the model from the saved model directory
model_dir = f"{PATH_TO_MODEL_DIR}/saved_model"
model = tf.saved_model.load(model_dir)

# Check the loaded model
print(f"Model loaded from: {model_dir}")


Model loaded from: C:/Users/ADVAIT/Desktop/CrashCal/models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model


In [5]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


In [6]:
import numpy as np
import cv2  # Ensure you have OpenCV installed for image display and saving

# Path to your COCO annotations and image directory
annotations_file = r'C:/Users/ADVAIT/Desktop/CarDD_release/CarDD_release/CarDD_COCO/annotations/instances_train2017.json'  # Path to annotations file
images_dir = r'C:/Users/ADVAIT/Desktop/CarDD_release/CarDD_release/CarDD_COCO/train2017'  # Path to images directory

# Path to your model directory and label map
PATH_TO_MODEL_DIR = r"C:/Users/ADVAIT/Desktop/CrashCal/models/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8"  # Model directory path
PATH_TO_LABELS = r'C:/Users/ADVAIT/Desktop/CrashCal/label_map.pbtxt'  # Path to the label map file

In [7]:
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [600, 600])  # Resize to a suitable input size
    image = image / 255.0  # Normalize to [0, 1]
    image = tf.cast(image * 255.0, tf.uint8)  # Convert back to [0, 255] range and change to uint8
    return image

# Run inference on images
def run_inference(model, image):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]  # Add batch dimension
    output_dict = model(input_tensor)  # Run the image through the model

    # Convert outputs to usable formats
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key: value[0, :num_detections].numpy() for key, value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # Handle detection classes and scores
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(int)
    return output_dict


In [8]:
def classify_damage(severity_score):
    if severity_score >= 0.8:
        return "Major"
    elif severity_score >= 0.5:
        return "Moderate"
    else:
        return "Minor"


In [9]:
def visualize_predictions(image, output_dict, coco_to_custom_mapping, threshold=0.5):
    # Define a color map for different classes
    color_map = {
        1: (0, 255, 0),  # Green for 'tire flat'
        3: (255, 0, 0),  # Blue for 'dent'
        8: (0, 0, 255),  # Red for 'scratch'
        70: (255, 255, 0),  # Cyan for 'crack'
        84: (255, 0, 255),  # Magenta for 'glass shatter'
        85: (0, 255, 255),  # Yellow for 'lamp broken'
        # Add any additional mappings as needed
    }

    # Filter detections based on confidence threshold
    scores = output_dict['detection_scores']
    boxes = output_dict['detection_boxes']
    classes = output_dict['detection_classes']

    # Prepare the image for visualization
    image_np = image.numpy() * 255.0  # Convert back to original range
    image_np = image_np.astype(np.uint8)  # Convert to uint8 for OpenCV

    for i in range(output_dict['num_detections']):
        if scores[i] > threshold:  # Only visualize detections above the threshold
            box = boxes[i]
            class_id = classes[i]
            score = scores[i]

            # Convert normalized box coordinates to pixel values
            h, w, _ = image_np.shape
            ymin, xmin, ymax, xmax = (box * np.array([h, w, h, w])).astype(int)

            # Determine color for the bounding box
            color = color_map.get(class_id, (255, 255, 255))  # Default to white if class ID is not found

            # Draw bounding box
            cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), color, 2)

            # Use mapping to get the category name
            category_name = coco_to_custom_mapping.get(class_id, "Unknown")
            label = f"{category_name} {score:.2f}"

            # Print label and score to terminal
            print(label)  # This will print to the terminal

            # Calculate text size and position for centering
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            text_x = xmin + (xmax - xmin - text_width) // 2  # Center horizontally
            text_y = ymin - 10 if ymin - 10 > text_height else ymin + text_height + 5  # Adjust if there's not enough space

            # Draw the text centered above the bounding box
            cv2.putText(image_np, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image_np


In [10]:
# Define the mapping from COCO class IDs to your custom categories
coco_to_custom_mapping = {
    1: 'lamp broken',
    3: 'dent',
    8: 'scratch',
    70: 'crack',
    84: 'glass shatter',
    85: 'tire flat',
    # Add any additional mappings as needed
}

# Use the new mapping in your visualization function
import numpy as np

# Loop through images, run inference, and visualize results
for img_path in images:
    img = preprocess_image(img_path)  # Preprocess the image
    output_dict = run_inference(model, img)  # Get predictions
    img_with_boxes = visualize_predictions(img, output_dict, coco_to_custom_mapping)  # Visualize the result

    # Create a blank canvas for padding
    height, width, _ = img_with_boxes.shape
    top_margin = 100  # Set your desired top margin
    canvas = np.zeros((height + top_margin, width, 3), dtype=np.uint8)  # Create a blank canvas
    canvas[top_margin:, :] = img_with_boxes  # Place the image on the canvas

    # Create a resizable window and center it
    cv2.namedWindow("Detected Objects", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Detected Objects", 800, 600)  # Resize as needed
    screen_res = (1920, 1080)  # Adjust based on your screen resolution
    cv2.moveWindow("Detected Objects", int(screen_res[0]/2 - 400), int(screen_res[1]/2 - 300))  # Center the window

    cv2.imshow("Detected Objects", canvas)  # Display the canvas with the image
    key = cv2.waitKey(0)  # Wait for a key press
    if key == ord('q'):  # If 'q' is pressed, break the loop
        break

cv2.destroyAllWindows()  # Close all OpenCV windows


crack 0.81
dent 0.54
dent 0.90
scratch 0.79


KeyboardInterrupt: 

In [14]:
# Specify the path to your custom image
img_path = r'C:/Users/ADVAIT/Desktop/03630.jpg'  # Change this to your image path

# Preprocess the image
img = preprocess_image(img_path)

# Run inference on the image
output_dict = run_inference(model, img)

# Visualize the results
img_with_boxes = visualize_predictions(img, output_dict, coco_to_custom_mapping)

# Create a blank canvas for padding
height, width, _ = img_with_boxes.shape
top_margin = 100  # Set your desired top margin
canvas = np.zeros((height + top_margin, width, 3), dtype=np.uint8)  # Create a blank canvas
canvas[top_margin:, :] = img_with_boxes  # Place the image on the canvas

# Create a resizable window and center it
cv2.namedWindow("Detected Objects", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Detected Objects", 800, 600)  # Resize as needed
screen_res = (1920, 1080)  # Adjust based on your screen resolution
cv2.moveWindow("Detected Objects", int(screen_res[0]/2 - 400), int(screen_res[1]/2 - 300))  # Center the window

# Display the canvas with the image
cv2.imshow("Detected Objects", canvas)
cv2.waitKey(0)  # Wait for a key press
cv2.destroyAllWindows()  # Close all OpenCV windows


scratch 0.95
dent 0.76
lamp broken 0.73
Unknown 0.66
dent 0.56
